In [1]:
# Import required libraries for Reddit API data collection
import praw  # For making HTTP requests to Reddit API
import pandas as pd  # For data manipulation and analysis
from datetime import datetime
from pathlib import Path

/opt/anaconda3/envs/uoft_senti_db/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
def get_reddit_instance():
    reddit = praw.Reddit(
        client_id = 'LDPs59MIc82xLXPftHQ3Sw',
        client_secret = "uZSSG2q3ueggUkUReIMzaKe8xnC6iw",
        password = 'Linyihao.041228',
        user_agent = "Comment Extraction (by SaltyfishQAQ)",
        username = 'Few-Strength-2343'
    )
    
    return reddit


def get_monthly_top_posts(reddit, subreddit_name, limit=10, time_filter='month'):
    count = 0
    # Fetch the subreddit
    subreddit = reddit.subreddit(subreddit_name)
    
    master_df = []
    all_submissions = []
    for submission in subreddit.top(time_filter=time_filter, limit=limit):
        single_submission = []
        # Collect submission information
        submission_info = {
            "submission_title": submission.title,
            "author": submission.author.name if submission.author else "deleted",
            "id": submission.id,
            "url": submission.url,
            "created_utc": submission.created_utc,
            "body": submission.selftext,  
            "type": "submission",
            "score": submission.score
        }
        master_df.append(submission_info)
        single_submission.append(submission_info)
        
        if submission.num_comments == 0:
            continue
        else:
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                # Skip MoreComments objects if any remain
                if isinstance(comment, praw.models.MoreComments):
                    continue
                
                comment_info = {
                    "submission_title": submission.title,
                    "author": comment.author.name if comment.author else "deleted",
                    "id": comment.id,
                    "url": f"https://www.reddit.com{comment.permalink}",
                    "created_utc": comment.created_utc,
                    "body": comment.body,
                    "type": "comment",
                    "score": comment.score
                }
                master_df.append(comment_info)
                single_submission.append(comment_info)
        
        all_submissions.append(single_submission)
        count += 1
        print(f"Processed submission {count}/{limit}: {submission.title}")
        
    master_df = pd.DataFrame(master_df).reset_index(drop=True)
        
    return master_df, all_submissions

In [3]:
reddit = get_reddit_instance()
master_df, all_submissions = get_monthly_top_posts(reddit, 'UofT', limit=100, time_filter='month')


Processed submission 1/100: these types of questions are actually so dumb what
Processed submission 2/100: PROPOSAL: Place a giant dinosaur statue on the lawn
Processed submission 3/100: 1st years on uoft Reddit after reading their replies
Processed submission 4/100: WELCOME BACK TO ANOTHER EPISODE OF STFU IN CLASS FOR THE LORD’S SAKE
Processed submission 5/100: Re: that post about putting a dinosaur on front campus
Processed submission 6/100: Okay UofT, I understand that the career fair was postponed, you can stop sending emails
Processed submission 7/100: My honest meme recreation of being a first year student so far
Processed submission 8/100: Recently diagnosed with stage 3 lung cancer, What do I tell my prof or ta?
Processed submission 9/100: Dr Ruth Marshall is now "on leave" following her comments in response to the murder of Charlie Kirk; university actively investigating the situation
Processed submission 10/100: I cried in the Robarts 3rd floor washroom — here’s how it helped

In [4]:

folder_path = Path(f'monthly_top100/{datetime.now().strftime("%Y%m")}')
folder_path.mkdir(parents=True, exist_ok=True)
for i, submission in enumerate(all_submissions):
    submission_df = pd.DataFrame(submission).reset_index(drop=True)
    submission_df.to_csv(folder_path / f'submission_{i}.csv', index=False)



    

In [5]:
top100_filename = f'top_100_reddits_{datetime.now().strftime("%Y%m")}.csv'
master_df.to_csv(folder_path / top100_filename)

In [7]:
master_df.shape

(2649, 8)